# Assignment 3: NLP 

# By: Michael Church Carson (260683849)


## import packages

In [1]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np

## Problem 1 The Markowitz Problem

## import problem data

In [2]:
Expected_RoRs = pd.read_excel('Markowitz.xlsx', sheet_name='Markowitz',usecols = 'L:P', nrows = 1)
Expected_RoRs

,FTSE 100 ERoR,DAX ERoR,DJIA ERoR,DJ Asian Titans 50 ERoR,Russell 2000 ERoR
0,0.069488,0.640171,0.693084,-0.107527,0.687519


In [3]:
SD_of_Returns = pd.read_excel('Markowitz.xlsx', sheet_name='Markowitz',usecols = 'Q:U', nrows = 1)
SD_of_Returns

,FTSE 100 SD,DAX SD,DJIA SD,DJ Asian Titans 50 SD,Russell 2000 SD
0,3.254885,5.286076,3.288999,4.522306,4.633412


In [4]:
Correlation = pd.read_excel('Markowitz.xlsx', sheet_name='CorrMatrix',usecols = 'A:E', nrows = 6)
Correlation

,FTSE 100,DAX,DJIA,DJ Asian Titans 50,Russell 2000
0,1.000000,0.755267,0.772833,0.687467,0.649829
1,0.755267,1.000000,0.703816,0.668050,0.694259
2,0.772833,0.703816,1.000000,0.721454,0.848499
3,0.687467,0.668050,0.721454,1.000000,0.670301
4,0.649829,0.694259,0.848499,0.670301,1.000000


In [5]:
ER = np.array(Expected_RoRs)
ER

array([[ 0.06948774,  0.64017118,  0.69308406, -0.10752718,  0.68751935]])

In [6]:
SD = np.array(SD_of_Returns)
SD

array([[3.25488488, 5.28607558, 3.28899906, 4.52230619, 4.63341247]])

In [7]:
Corr = np.array(Correlation)
Corr

array([[1.        , 0.75526686, 0.77283301, 0.68746749, 0.64982921],
       [0.75526686, 1.        , 0.70381606, 0.66804974, 0.69425926],
       [0.77283301, 0.70381606, 1.        , 0.72145449, 0.8484989 ],
       [0.68746749, 0.66804974, 0.72145449, 1.        , 0.67030134],
       [0.64982921, 0.69425926, 0.8484989 , 0.67030134, 1.        ]])

In [8]:
Assets = ["FTSE_100","DAX","DJIA", "DJ Asian Titans 50", "Russell 2000"]
a = len(Assets)

In [9]:
Cov_mat=(SD * Corr * SD)

In [10]:
Cov_mat

array([[10.59427558, 21.10411607,  8.36013255, 14.05957171, 13.95086562],
       [ 8.00150528, 27.94259502,  7.61354061, 13.66245451, 14.90471256],
       [ 8.18760592, 19.66644706, 10.81751479, 14.75464845, 18.21600805],
       [ 7.28322001, 18.66704344,  7.80434458, 20.45125328, 14.39037175],
       [ 6.88446974, 19.39940526,  9.1786494 , 13.70850248, 21.4685111 ]])

## P1 (a)

## Initiate Model

In [11]:
prob = gp.Model("Markowitz Problem.")

Academic license - for non-commercial use only - expires 2022-09-24
Using license file /Users/michaelchurchcarson/gurobi.lic


## Define the decision variables and the objective function

In [12]:
Vars=prob.addMVar(a,lb=0,vtype=GRB.CONTINUOUS,name=['portfolio proportion in_'+i for i in Assets])

In [13]:
#Expected return of the portfolio
P_ER=ER@Vars

In [14]:
#Variance of the portfolio
P_Var=(Vars @ Cov_mat @ Vars)

In [15]:
prob.setObjective(P_Var,GRB.MINIMIZE)

## Define the contraints
Introduce portflio allocation constraint - the sum of the the proportion invested in each asset must be 1, and
introduce non-negativite return constraint.

In [16]:
prob.addConstr(sum(Vars) == 1)
prob.addConstr(P_ER>=0)

<(1,) matrix constraint *awaiting model update*>

## Find the optimal solution

In [17]:
prob.setParam('OutputFlag', 0)
prob.optimize()

## Show the optimal solution

Recall: we minimized the variance of the portfolio, but we need to minimize standard deviation. Therefore, to get the optimal standard deviation value we take the square root of the optimal solution to our minimization problem.

In [18]:
(prob.ObjVal)**0.5

3.0801494736128516

## Show the optimal investment in each asset (index) to minimize risk (standard deviation)

In [19]:
import pandas as pd
Vars.X

minrisk_volatility = (prob.ObjVal)**0.5
pd.DataFrame(data=np.append(Vars.X, [minrisk_volatility]),
             index=Assets + ['Volatility'],
             columns=['Minimum Risk Portfolio'])

,Minimum Risk Portfolio
FTSE_100,5.229479e-01
DAX,2.941485e-12
DJIA,4.770521e-01
DJ Asian Titans 50,1.520668e-10
Russell 2000,1.779441e-11
Volatility,3.080149e+00


### Show the optimal investment in each asset (index) to minimize risk (standard deviation) and round the allocation so that the very small investments in DAX, DJ Asian Titans 50 and Russell 2000 show as zero (justification for this is provided in the report)

In [20]:
import pandas as pd
Vars.X

minrisk_volatility = (prob.ObjVal)**0.5
pd.DataFrame(data=np.append((np.round(Vars.X,4))*100, [minrisk_volatility]),
             index=Assets + ['Volatility'],
             columns=['Minimum Risk Portfolio'])

#print(Vars[i+1].varName,': ', round(Vars[i+1].x*100,2),'%')

,Minimum Risk Portfolio
FTSE_100,52.290000
DAX,0.000000
DJIA,47.710000
DJ Asian Titans 50,0.000000
Russell 2000,0.000000
Volatility,3.080149


## P1 (b)

### Re-import the data so that it can be imported without using matrices (arrays) because using the matrices from the P1 (a) was leading to complications with the new constraint we need for this problem to incorporate transaction costs

In [21]:
# Import entire Markowitz excel as a dataframe 
data=pd.read_excel('Markowitz2.xlsx', sheet_name='Markowitz')

# Clean and sort data in the dataframe
data['Date']=pd.to_datetime(data['Date'], dayfirst=True)

data=data.sort_values(by=['Date'],ignore_index=True)

data.drop(data.columns[0], axis=1, inplace=True)


#Recall
#Assets = ["FTSE_100","DAX","DJIA", "DJ Asian Titans 50", "Russell 2000"]
#a = len(Assets)

## Calculate rate of return (RoR)

In [22]:
RoR = pd.DataFrame(columns = data.columns)
for colname in data.columns:
    RoR[colname]=data[colname].pct_change()
    
RoR=RoR.drop(RoR.index[0]).reset_index()
RoR.drop(RoR.columns[0], axis=1, inplace=True)

In [23]:
# Rate of returns mean
means={}
i=1
for colname in RoR.columns:
    means[i]=RoR[colname].mean()
    i=i+1
# Rate of returns covariance matrix
cov_mat=RoR.cov()

# Now we annualize the means and covariance
# Annual mean
annual_mean={}
for i in range(len(RoR.columns)):
    annual_mean[i+1]=((1+means[i+1])**12)-1

# Covariance
annual_cov=cov_mat*12

## Initiate Model

In [24]:
prob1 = gp.Model("Markowitz Problem w Transaction Costs")

## Define the decision variables

In [25]:
assets={}
i=1
for asset in RoR.columns:
    assets[i]=prob1.addVar(lb = 0, ub=1, name = asset)
    i=i+1

# Introduce a DV to use in our objective functions
X = prob1.addVar(lb = 0, name = 'Dummy')

## Define the contraints

In [26]:
# Quadratic constraint
prob1.addConstr((sum(assets[i+1]*assets[j+1]*annual_cov.iloc[i,j] for i in range(a) for j in range(a)))<=X)

# Budget constraint
prob1.addConstr((sum(assets[i+1] for i in range(a)))==1)

# Non-negative return constraint
prob1.addConstr((sum((assets[i+1]*annual_mean[i+1])-((assets[i+1]-0.20)*(assets[i+1]-0.20)) for i in range(a)))>=0)

<gurobi.QConstr Not Yet Added>

## Define the objective function

In [27]:
exp = X
prob1.setObjective(exp, GRB.MINIMIZE)

## Find the optimal solution

In [28]:
prob1.setParam('OutputFlag', 0)
prob1.optimize()

## Show the optimal solution

In [29]:
print('\nThe risk for the optimal solution is:',round(prob1.objVal,4))


The risk for the optimal solution is: 0.0137


## Show the optimal investment in each asset (index) to minimize risk (standard deviation)

In [30]:
print("\nThe optimal investement in each asset is:")
for i in range(len(assets)):
    print(assets[i+1].varName,': ', round(assets[i+1].x*100,2),'%')


The optimal investement in each asset is:
FTSE 100 :  31.11 %
DAX :  7.84 %
DJIA :  31.92 %
DJ Asian Titans 50 :  14.34 %
Russell 2000 :  14.79 %


## Problem 2 The CYCOM Corporation Problem

## P2 (a)

## Initiate Model

In [52]:
model = gp.Model("CYCOM Corp.")

## Define the decision variables and the objective function

In [53]:
x1 = model.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'x1')
x2 = model.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'x2')
x3 = model.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'x3')
x4 = model.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'x4')
x5 = model.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'x5')
x6 = model.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'x6')

p1 = model.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'p1')
p2 = model.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'p2')
p3 = model.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'p3')
p4 = model.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'p4')
p5 = model.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'p5')
p6 = model.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'p6')

## Expected profit

In [54]:
EP = 1750*(p1) +700*(p2) + 1300*(p3) + 800*(p4) + 1450*(p5) + 1300*(p6)

## Expected start-up cost

In [55]:
ES = 325*(1-p1) + 200*(1-p2) + 490*(1-p3) + 125*(1-p4) + 710*(1-p5) + 240*(1-p6)

In [56]:
exp = EP - ES - 150*(x1 + x2 + x3 + x4 + x5 + x6)

In [57]:
model.setObjective(exp, GRB.MAXIMIZE)
model.params.NonConvex = 2

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1


## Define the contraints

In [58]:
engineer_availabilty = model.addConstr(x1 + x2 + x3 + x4 + x5 + x6 <= 25)

In [59]:
Constraint_x1 = model.addConstr(x1 == p1*(x1+1.1))
Constraint_x2 = model.addConstr(x2 == p2*(x2+0.5))
Constraint_x3 = model.addConstr(x3 == p3*(x3+2.5))
Constraint_x4 = model.addConstr(x4 == p4*(x4+1.6))
Constraint_x5 = model.addConstr(x5 == p5*(x5+2.2))
Constraint_x6 = model.addConstr(x6 == p6*(x6+2.4))

## Find the optimal solution

In [60]:
model.setParam('OutputFlag', 0)
model.optimize()

## Show the optimal solution

In [61]:
round(model.objVal, 5)*1000

1396484.34

## Show the optimal values for the DVs

In [62]:
for var in model.getVars():
        print(var.varName, "=", round(var.x,1))

x1 = 2.8
x2 = 1.2
x3 = 3.0
x4 = 1.5
x5 = 3.4
x6 = 2.6
p1 = 0.7
p2 = 0.7
p3 = 0.5
p4 = 0.5
p5 = 0.6
p6 = 0.5


## P2 (b)

## Initiate Model

In [63]:
model1 = gp.Model("CYCOM Corp.")

## Define the decision variables and the objective function

In [64]:
x1 = model1.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'x1')
x2 = model1.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'x2')
x3 = model1.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'x3')
x4 = model1.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'x4')
x5 = model1.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'x5')
x6 = model1.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'x6')

p1 = model1.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'p1')
p2 = model1.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'p2')
p3 = model1.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'p3')
p4 = model1.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'p4')
p5 = model1.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'p5')
p6 = model1.addVar(vtype= GRB.CONTINUOUS, lb = 0, name = 'p6')

In [65]:
## Expected profit

EP = 1750*(p1) +700*(p2) + 1300*(p3) + 800*(p4) + 1450*(p5) + 1300*(p6)

## Expected start-up cost

ES = 325*(1-p1) + 200*(1-p2) + 490*(1-p3) + 125*(1-p4) + 710*(1-p5) + 240*(1-p6)

In [66]:
## Variance of contribution to profit

VP = (1750+325)**2*p1*(1-p1) + (700+200)**2*p2*(1-p2) + (1300+490)**2*p3*(1-p3) + (800+125)**2*p4*(1-p4) + (1450+710)**2*p5*(1-p5) + (1300+240)**2*p6*(1-p6)

In [67]:
exp = VP

In [68]:
model1.setObjective(exp, GRB.MINIMIZE)
model1.params.NonConvex = 2

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1


## Define the contraints

In [69]:
engineer_availabilty = model1.addConstr(x1 + x2 + x3 + x4 + x5 + x6 <= 25)

In [70]:
min_contribution = model1.addConstr(EP - ES - 150*(x1 + x2 + x3 + x4 + x5 + x6) >= 1100)

In [71]:
Constraint_x1 = model1.addConstr(x1 == p1*(x1+1.1))
Constraint_x2 = model1.addConstr(x2 == p2*(x2+0.5))
Constraint_x3 = model1.addConstr(x3 == p3*(x3+2.5))
Constraint_x4 = model1.addConstr(x4 == p4*(x4+1.6))
Constraint_x5 = model1.addConstr(x5 == p5*(x5+2.2))
Constraint_x6 = model1.addConstr(x6 == p6*(x6+2.4))

## Find the optimal solution

In [72]:
model1.setParam('OutputFlag', 0)
model1.optimize()

## Show the optimal solution

## Recall that we now need to take the square root of the optimal variance value to get the optimal standard deviation

In [73]:
round(((model1.objVal)**0.5),2)

1800.36

## Show the optimal values for the DVs

In [74]:
for var in model1.getVars():
        print(var.varName, "=", round(var.x,1))

x1 = 5.1
x2 = 1.6
x3 = 4.2
x4 = 1.5
x5 = 6.0
x6 = 3.0
p1 = 0.8
p2 = 0.8
p3 = 0.6
p4 = 0.5
p5 = 0.7
p6 = 0.6


## END